# Linear regression

In [1]:
using Flux.Tracker
using DelimitedFiles: readdlm
using Statistics: mean, std

## Download data

In [2]:
isfile("housing.data") ||
  download("https://raw.githubusercontent.com/MikeInnes/notebooks/master/housing.data", "housing.data")

true

## Read data

In [3]:
rawdata = readdlm("housing.data")'

14×505 LinearAlgebra.Adjoint{Float64,Array{Float64,2}}:
   0.02731    0.02729    0.03237  …    0.06076    0.10959    0.04741
   0.0        0.0        0.0           0.0        0.0        0.0    
   7.07       7.07       2.18         11.93      11.93      11.93   
   0.0        0.0        0.0           0.0        0.0        0.0    
   0.469      0.469      0.458         0.573      0.573      0.573  
   6.421      7.185      6.998    …    6.976      6.794      6.03   
  78.9       61.1       45.8          91.0       89.3       80.8    
   4.9671     4.9671     6.0622        2.1675     2.3889     2.505  
   2.0        2.0        3.0           1.0        1.0        1.0    
 242.0      242.0      222.0         273.0      273.0      273.0    
  17.8       17.8       18.7      …   21.0       21.0       21.0    
 396.9      392.83     394.63        396.9      393.45     396.9    
   9.14       4.03       2.94          5.64       6.48       7.88   
  21.6       34.7       33.4          23.9     

## The last feature (the price of the house) is our target

In [4]:
x = rawdata[1:13,:]
y = rawdata[14:14,:]

1×505 Array{Float64,2}:
 21.6  34.7  33.4  36.2  28.7  22.9  …  16.8  22.4  20.6  23.9  22.0  11.9

## Normalization

$\LARGE z = \frac{x - \mu}{\sigma}$

In [5]:
x = (x .- mean(x; dims=2)) ./ std(x; dims=2)

13×505 Array{Float64,2}:
 -0.417416  -0.417418  -0.416828  …  -0.41353   -0.407858  -0.415081
 -0.486234  -0.486234  -0.486234     -0.486234  -0.486234  -0.486234
 -0.595732  -0.595732  -1.30899       0.11315    0.11315    0.11315 
 -0.272618  -0.272618  -0.272618     -0.272618  -0.272618  -0.272618
 -0.739098  -0.739098  -0.833934      0.15753    0.15753    0.15753 
  0.194741   1.28121    1.01528   …   0.983996   0.725177  -0.361293
  0.366208  -0.265527  -0.808535      0.795646   0.735312   0.433641
  0.556346   0.556346   1.0759       -0.771891  -0.66685   -0.611768
 -0.868939  -0.868939  -0.754097     -0.983782  -0.983782  -0.983782
 -0.987128  -0.987128  -1.10573      -0.803294  -0.803294  -0.803294
 -0.306024  -0.306024   0.110158  …   1.17373    1.17373    1.17373 
  0.441136   0.396591   0.416291      0.441136   0.403377   0.441136
 -0.494157  -1.20985   -1.36251      -0.984357  -0.866709  -0.670629

## Model

In [6]:
predict(x) = W*x .+ b

predict (generic function with 1 method)

In [7]:
W = param(randn(1,13)/10)
b = param([0.])

Tracked 1-element Array{Float64,1}:
 0.0

## *Using CUDA*

```julia
using CuArrays
W, b, x, y = cu.((W, b, x, y))
```

## Loss function

$\LARGE \frac{\sum (\hat{y} - y)^2}{n}$

In [8]:
error(ŷ, y) = sum((ŷ .- y).^2)/size(y, 2)

error (generic function with 1 method)

In [9]:
loss(x, y) = error(predict(x), y)

loss (generic function with 1 method)

## Optimizer (gradient descent)

In [10]:
function update!(ps, η = .1)
    for w in ps
        w.data .-= w.grad .* η
        w.grad .= 0
    end
end

update! (generic function with 2 methods)

## Training

In [11]:
for i = 1:50
    back!(loss(x, y))
    update!((W, b))
    @show loss(x, y)
end

loss(x, y) = 367.24844435760315 (tracked)
loss(x, y) = 241.9634770067354 (tracked)
loss(x, y) = 163.12434930585846 (tracked)
loss(x, y) = 112.86873071697995 (tracked)
loss(x, y) = 80.76150714894602 (tracked)
loss(x, y) = 60.21774822234975 (tracked)
loss(x, y) = 47.05082158083555 (tracked)
loss(x, y) = 38.5946045023392 (tracked)
loss(x, y) = 33.14951403626839 (tracked)
loss(x, y) = 29.631148287794435 (tracked)
loss(x, y) = 27.347054603796384 (tracked)
loss(x, y) = 25.854723348804853 (tracked)
loss(x, y) = 24.871156259747725 (tracked)
loss(x, y) = 24.215225414564006 (tracked)
loss(x, y) = 23.770903553292584 (tracked)
loss(x, y) = 23.463791905843443 (tracked)
loss(x, y) = 23.246127157963414 (tracked)
loss(x, y) = 23.087199055598624 (tracked)
loss(x, y) = 22.96722280830853 (tracked)
loss(x, y) = 22.873418706456736 (tracked)
loss(x, y) = 22.79750261811842 (tracked)
loss(x, y) = 22.734078768362085 (tracked)
loss(x, y) = 22.67960980409478 (tracked)
loss(x, y) = 22.63175637166298 (tracked)
los

## Prediction

In [12]:
predict(x[:, 1]) / y[1]

Tracked 1-element Array{Float64,1}:
 1.1568449952894109